In [2]:
#Import drive
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/Colab Notebooks/Sebastian

/content/drive/MyDrive/Colab Notebooks/Sebastian


Stuff for commiting to Github below

In [10]:
ROOT = '/content/drive"'

In [45]:
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'drive/MyDrive/Colab Notebooks/Sebastian' 
# replace with your Github username 
GIT_USERNAME = "ahhuang007" 
# definitely replace with your
GIT_TOKEN = "new token"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "Sebastian" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path     

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
#GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
#print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive"/drive/MyDrive/Colab Notebooks/Sebastian


In [ ]:
!git add .

In [ ]:
!git config --global user.email "ahhuang007@gmail.com"
!git config --global user.name "ahhuang007"

In [13]:
!git commit -m "changed the done condition and removed survival reward"

[main 47464bf] changed the done condition and removed survival reward
 7 files changed, 5 insertions(+), 3 deletions(-)
 rewrite colab.ipynb (95%)
 rewrite random_model_ppo.zip (79%)
 rewrite real_model_ppo.zip (90%)


In [53]:
!git remote set-url origin https://$git_key@github.com/ahhuang007/Sebastian.git

Had to reset the remote origin URL here, removed the code since it contained the personal authorization token. Now I set the git key as an environment variable.

--------

Command for pulling from github

In [4]:
!git pull origin

Already up to date.


In [14]:
!git push origin

Counting objects: 14, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (14/14), done.
Writing objects: 100% (14/14), 214.47 KiB | 8.58 MiB/s, done.
Total 14 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/ahhuang007/Sebastian.git
   262ece7..47464bf  main -> main


Getting OpenAI Baselines

In [ ]:
!git clone https://github.com/openai/baselines.git

Cloning into 'baselines'...
remote: Enumerating objects: 3627, done.
remote: Total 3627 (delta 0), reused 0 (delta 0), pack-reused 3627
Receiving objects: 100% (3627/3627), 6.46 MiB | 8.34 MiB/s, done.
Resolving deltas: 100% (2429/2429), done.
Checking out files: 100% (199/199), done.


Baselines doesn't come installed, tragic

In [ ]:
cd baselines

/content/drive/MyDrive/Colab Notebooks/Sebastian/baselines


In [ ]:
!pip install -e .

Installing stable_baselines since it might be better

In [ ]:
%cd ..

/content/drive/MyDrive/Colab Notebooks


In [4]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 174 kB 5.2 MB/s 


Installing my custom environment

In [1]:
cd Sebastian

[Errno 2] No such file or directory: 'Sebastian'
/content


In [5]:
%cd gym-seb

/content/drive/MyDrive/Colab Notebooks/Sebastian/gym-seb


In [6]:
!pip install -e .

Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/Sebastian/gym-seb
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 89.3 MB 63 kB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=a2060031a01bb1d936ee55e5425c9c6152975bed0b8987c33ed8bad3fe5f1ad3
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
  Running setup.py develop for gym-seb
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 1.3.0 requires gym<0.20,>=0.17, but you have gym 0.21.0 which is incompatible.


Restart runtime here

In [1]:
%cd drive/My Drive/Colab Notebooks/Sebastian

/content/drive/My Drive/Colab Notebooks/Sebastian


In [2]:
from stable_baselines3 import DDPG, PPO
import gym

In [3]:
import gym_seb
env = gym.make('seb-v0', max_timesteps = 10000, use_gui = False)
print(env.observation_space)

Box([-100000. -100000. -100000. -100000. -100000. -100000. -100000. -100000.
 -100000. -100000. -100000. -100000. -100000. -100000. -100000. -100000.
 -100000. -100000. -100000. -100000. -100000. -100000. -100000. -100000.
 -100000. -100000. -100000.], [100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000.
 100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000.
 100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000. 100000.], (27,), float32)


In [4]:
from stable_baselines3.common.env_checker import check_env

check_env(env, warn=True)

resetting environment


In [5]:
from stable_baselines3.ddpg.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

'''
model finished running when "doneness" was achievable - do i need to do this?
I think the model will keep learning until it is done - if it is done but 
has not reached the number of total timesteps, it will reset and learn again
until it has surpassed the total_timesteps and is done again
'''

model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
from stable_baselines3.common.evaluation import evaluate_policy

#performance with random model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


resetting environment
at episode 10000
(0.006357660762663624, -0.011317405243910983, 0.08492829342543552)
max timesteps reached
resetting environment
at episode 10000
(0.00040838613088514127, 0.0075819800381330465, 0.08501277955617964)
max timesteps reached
resetting environment
at episode 10000
(0.009620931331744425, -0.010620140883110701, 0.08485501367151274)
max timesteps reached
resetting environment
at episode 10000
(-0.0018692742562436584, -0.0068477344375425485, 0.0849925049607279)
max timesteps reached
resetting environment
at episode 10000
(0.0013722602912573266, -0.0004267816023497047, 0.08507882029977862)
max timesteps reached
resetting environment
at episode 10000
(0.0007120627081942846, -0.0005733045365859217, 0.08507209212869556)
max timesteps reached
resetting environment
at episode 10000
(0.000596503185311076, -0.0005317622151795746, 0.0850731738205799)
max timesteps reached
resetting environment
at episode 10000
(0.0005613307105754306, -0.0004909413885837474, 0.0850752

In [7]:
model.save("random_model_ppo")

In [8]:
env.reset()
#set total_timesteps equal to n_eval_episodes * max_timesteps
#Setting for loop so we can train in pieces in case colab kicks us off

model = model.learn(total_timesteps = 200000, n_eval_episodes = 20, log_interval = 10000)
model.save("real_model_ppo")

resetting environment
resetting environment
robot has flipped over at timestep 1122
resetting environment
robot has flipped over at timestep 972
resetting environment
robot has flipped over at timestep 323
resetting environment
robot has flipped over at timestep 184
resetting environment
robot has flipped over at timestep 528
resetting environment
robot has flipped over at timestep 138
resetting environment
robot has flipped over at timestep 247
resetting environment
robot has flipped over at timestep 375
resetting environment
robot has flipped over at timestep 226
resetting environment
robot has flipped over at timestep 269
resetting environment
robot has flipped over at timestep 179
resetting environment
robot has flipped over at timestep 465
resetting environment
robot has flipped over at timestep 1417
resetting environment
robot has flipped over at timestep 163
resetting environment
robot has flipped over at timestep 291
resetting environment
robot has flipped over at timestep 258


In [9]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


resetting environment
robot has flipped over at timestep 4545
resetting environment
robot has flipped over at timestep 8866
resetting environment
robot has flipped over at timestep 2152
resetting environment
robot has flipped over at timestep 5906
resetting environment
robot has flipped over at timestep 1537
resetting environment
robot has flipped over at timestep 2773
resetting environment
robot has flipped over at timestep 2965
resetting environment
robot has flipped over at timestep 2654
resetting environment
robot has flipped over at timestep 7642
resetting environment
robot has flipped over at timestep 2992
resetting environment
mean_reward=-5922.80 +/- 3282.836648929835


So, yeah, my algorithm's improving but it still needs to improve a lot more. It would appear that PPO is better than DDPG, so I think I'll be working with PPO from here on out.

Possible methods of improvement: Expanding observation space to include joint angles/velocities, contact forces, rotation matrix, increase reward based on velocity and magnitude of action taken, 